In [ ]:
import requests
import json
import csv
from types import SimpleNamespace

In [ ]:
#ToDo: don't just hardcode the subtypes
subtypeMapping = {
    0: "Install or New",
    1: "Service Request",
    2: "Incident",
}

def getSTId(subtype):
    for key, value in subtypeMapping.items():
        if value == subtype:
            return key
    return None

def getSubtype(STId):
    for key, value in subtypeMapping.items():
        if key == STId:
            return value
    return None

In [ ]:
def request_paginated(url, page):
    headers = {
    }
    furl = f"{url}&page={page}"
    response = requests.request("GET", furl, headers=headers)
    data = json.loads(response.text, object_hook=lambda d: SimpleNamespace(**d))
    return data

def request_count(url):
    headers = {
    }
    a = url.split("?")
    a[0] = a[0]+"count/"
    curl = "?".join(a)
    count = json.loads((requests.request("GET", curl, headers=headers)).text)["count"]
    return count

def write_to_file(data,writer):
    for d in data:
        label = getSTId(d.subType.name)
        text = d.summary
        val = (label,text)
        writer.writerow(val)

In [ ]:
pagesize = 1000
file = 'trainingData.csv'
fileHead = ('Label', 'Text')

url = f"REST URL with lots of conditions"

count = request_count(url)
iterations = count//pagesize
page = 0

with open(file, 'w', newline= '') as file:
    writer = csv.writer(file)
    writer.writerow(fileHead)
    while page < iterations:
        print(f"Processing page {page} of {iterations}")
        data = request_paginated(url, page)
        write_to_file(data,writer)
        page +=1